#**Import Modules**

Selenium, Openpyxl, Files, Time

In [6]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromiu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *

In [0]:
import time

In [0]:
import openpyxl

In [0]:
from google.colab import files

#**Mount Google Drive**

In [11]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [12]:
cd drive/My Drive/Colab Notebooks/Paper/Telecommunication

/content/drive/My Drive/Colab Notebooks/Paper/Telecommunication


In [13]:
ls

Crawler/  Data/  Paper-Keywords/


#**Fetch all Volume Links**

In [14]:
MAINURLFORMAT = "https://www.sciencedirect.com/journal/telecommunications-policy/issues?page="
volUrls = []

for i in range(3):
    # access page
    print("Reading, " + MAINURLFORMAT + str(i + 1))
    wd.get(MAINURLFORMAT + str(i + 1))
    time.sleep(0.5)

    # toggle all dropdown buttons
    btns = wd.find_elements_by_xpath("//button[@aria-expanded]")
    for btn in btns:
        if "accordion" in btn.get_attribute("id"):
            while "true" not in btn.get_attribute("aria-expanded"):
                btn.click()
                time.sleep(0.5)

    # fetch volume urls
    a_tags = wd.find_elements_by_xpath("//a[@href]")
    for tag in a_tags:
        if "vol" in tag.get_attribute("href") and not tag.get_attribute("href") in volUrls:
            volUrls.append(tag.get_attribute("href"))

Reading, https://www.sciencedirect.com/journal/telecommunications-policy/issues?page=1
Reading, https://www.sciencedirect.com/journal/telecommunications-policy/issues?page=2
Reading, https://www.sciencedirect.com/journal/telecommunications-policy/issues?page=3


In [0]:
# check missing volume no.
tmps = []

for volurl in volUrls:
    tmp = volurl[volurl.index("vol") + 4:]
    tmp = tmp[:tmp.index("/")]
    if tmp not in tmps:
        tmps.append(tmp)

for i in range(44):
    if str(i + 1) not in tmps:
        print(i + 1)

#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "kwd"

In [0]:
TARGETPATH = "./Data/raw/International/"

In [17]:
 r = 2

 # write header
 ws.cell(row = 1, column = 1).value = 'title'
 ws.cell(row = 1, column = 2).value = 'journal'
 ws.cell(row = 1, column = 3).value = 'year'
 ws.cell(row = 1, column = 4).value = 'keywords'
 wb.create_sheet("auth")
 ws = wb["auth"]
 ws.cell(row = 1, column = 1).value = 'title'
 ws.cell(row = 1, column = 2).value = 'authors'
 ws = wb["kwd"]

 for volurl in volUrls:
    # access volume page
    wd.get(volurl)
    time.sleep(1)

    # fetch volume no. from url
    vol = volurl[volurl.index("vol") + 4:]
    vol = vol[:vol.index("/")]

    # fetch year
    try:
        yr = wd.find_element_by_xpath("//h3[@class='js-issue-status text-s']").get_attribute("innerText").split(" ")[-1].replace(")", "").replace("(", "").replace(" ", "")
    except NoSuchElementException:
        continue
    
    print("Reading Volume from Year, " + yr + " : "  + volurl)
    
    # fetch article urls
    articleUrls = []
    a_tags = wd.find_elements_by_xpath("//a[@href]")
    for tag in a_tags:
        if (("article/pii" in tag.get_attribute("href")) and (not "pdf" in tag.get_attribute("href")) and (not "Editorial Board" in tag.text) and (not tag.get_attribute("href") in articleUrls)):
            articleUrls.append(tag.get_attribute("href"))

    for articleurl in articleUrls:
        # access article page
        wd.get(articleurl)
        time.sleep(0.8)
        #print("Reading article, " + articleurl)

        # fetch authors
        authors = []
        auth_gn_tags = wd.find_elements_by_xpath("//span[@class='text given-name']")
        auth_sn_tags = wd.find_elements_by_xpath("//span[@class='text surname']")
        for j in range(len(auth_gn_tags)):
            authors.append(auth_gn_tags[j].text + " " + auth_sn_tags[j].text)
        if(len(authors) == 0):
            continue
        #print(authors)

        # fetch title
        try:
            title = wd.find_element_by_class_name("title-text").text
        except NoSuchElementException:
            continue
        #print(title)

        # fetch keywords
        keywords = []
        kwd_header = wd.find_elements_by_class_name("keywords-section")
        for candid in kwd_header:
            try:
                if(candid.find_element_by_xpath("./h2").text == "Keywords"):
                    kwd_tags = candid.find_elements_by_xpath("./div")
                    for kwd in kwd_tags:
                        keywords.append(kwd.text)
                else:
                    continue
            except NoSuchElementException:
                continue
        while '' in keywords:
            keywords.remove('')
        if(len(keywords) == 0):
            continue
        #print(keywords)

        # write data to excel file
        print("Writing, " + title)
        ws = wb["kwd"]
        ws.cell(row = r, column = 1).value = title
        ws.cell(row = r, column = 2).value = "Telecommunication Policy"
        ws.cell(row = r, column = 3).value = yr
        for j in range(len(keywords)):
            ws.cell(row = r, column = 4 + j).value = keywords[j]
        ws = wb["auth"]
        ws.cell(row = r, column = 1).value = title
        for j in range(len(authors)):
            ws.cell(row = r, column = 2 + j).value = authors[j]
        r += 1

    # backup for every ten volumes
    if(volUrls.index(volurl) % 20 == 0):
        print("Saving backup file : " + str(volUrls.index(volurl) + 1) + " out of " + str(len(volUrls)))
        wb.save(filename = TARGETPATH + "telecommunications_policy_backup.xlsx")
        time.sleep(20)
        #files.download(TARGETPATH + "telecommunications_policy_backup.xlsx")

print("Saving file")
wb.save(filename = TARGETPATH + "telecommunications_policy.xlsx")

Reading Volume from Year, 2020 : https://www.sciencedirect.com/journal/telecommunications-policy/vol/44/issue/3
Writing, Financial inclusion, mobile money, and individual welfare: The case of Burkina Faso
Writing, Opportunity cost of spectrum for mobile communications: Evaluation of spectrum prices in Bangladesh
Writing, Interoperability of interpersonal communications services – A consumer perspective
Writing, Investigating the intertwining impact of wholesale access pricing and the commitment to net neutrality principle on European next-generation access networks private investment plans: An options-game application for capturing market players' competitive interactions
Saving backup file : 1 out of 308
Reading Volume from Year, 2020 : https://www.sciencedirect.com/journal/telecommunications-policy/vol/44/issue/2
Writing, ICT as a driver of economic growth: A survey of the literature and directions for future research
Writing, Mainframes and mandarins: The impact of internet use on i

#**Save File**

In [0]:
wb.save(filename = TARGETPATH + "telecommunications_policy.xlsx")

#**Download FIle**

In [0]:
files.download(TARGETPATH + "telecommunications_policy.xlsx")